# 验证码识别测试

由于生成10000张图片的tfrecords耗费时间太长，先写入500张进行测试

In [ ]:
li = ['a','b','c','d']
print(li[1:3])

In [2]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import os
import numpy as np

In [ ]:
def getOnehot(label):
    # float64
    label1 = np.zeros((40,))
    for i in range(4):
        label1[i*10+label[i]]=1
    return label1

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
DIR = "./CaptchaImage/"

def decodeJPG(filename, sess, writer):
    image = tf.gfile.FastGFile(os.path.join(DIR, filename), 'rb').read()
    # 将图片解码，解成张量  image is a 3-D uint8 or uint16 Tensor of shape [height, width, channels]
    image = tf.image.decode_jpeg(image)
    # 转化为灰度图
    image = sess.run(tf.image.rgb_to_grayscale(image))
    # 将图片转化为1维
    # 这一步不run的话，返回tensor变量，run了之后返回ndarray
    image = sess.run(tf.reshape(image, [-1]))
    print(image.shape)
    # 获得标签
    names = filename.split('.')
    lab= sess.run(tf.reshape([int(i) for i in list(names[0])],[-1]))
    label = getOnehot(lab)
    # 将一个验证码图片转化为Example Protocol Buffer
    example = tf.train.Example(features=tf.train.Features(feature={
        'input_pixels_num': _int64_feature(160 * 60),
        'label': _bytes_feature(label.tostring()),
        'image': _bytes_feature(image.tostring())
    }))
    writer.write(example.SerializeToString())
   
          
filename = "./CaptchaRecognize/output.tfrecords"
          
with tf.Session() as sess:
    # 创建一个writer来写tfrecord文件
    with tf.python_io.TFRecordWriter(filename) as writer:
        for root, dirs, files in os.walk(DIR):
            # 定义进度条
            with tqdm(total=1000, desc='进度') as bar:
                for file in files[0:1000]:
                    decodeJPG(file, sess=sess, writer=writer)
                    bar.update(1)

进度:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
# 读取tfrecords文件
# 创建一个reader对象
reader = tf.TFRecordReader()
# 创建一个队列维护输入文件列表
filename_queue = tf.train.string_input_producer(["./CaptchaRecognize/output.tfrecords"])
_, serialized_example = reader.read(filename_queue)
# 使用parse_single_example 一次只取一个数据
# 使用parse_example 可以取多个
feature = tf.parse_single_example(serialized_example,
                                 features={
        'input_pixels_num':tf.FixedLenFeature([],tf.int64),
        'label': tf.FixedLenFeature([],tf.string),
        'image': tf.FixedLenFeature([],tf.string) 
                                 })
image = tf.decode_raw(feature['image'],tf.uint8)
label = tf.decode_raw(feature['label'],tf.float64)
# label = tf.reshape(tf.cast(label,tf.float32), [-1])
pixnum = feature['input_pixels_num']
sess = tf.Session()
# # 启动多线程处理输入数据
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

import matplotlib.pyplot as plt
for i in range(5):
#     img = sess.run(image)
#     im = sess.run(tf.reshape(img,[60,160]))
#     plt.figure(1)
#     plt.imshow(im)
#     plt.show()
   print(sess.run([image,label]))

# sess.close() 上面程序应该是自动关闭了sess，手动关闭会报错

# 创建batch读取数据

In [ ]:
image = tf.decode_raw(feature['image'],tf.uint8)
image.set_shape([9600])
image = tf.cast(image,tf.float32)
label = tf.decode_raw(feature['label'],tf.float64)
label.set_shape([40])
label = tf.cast(label,tf.float32)
# sess = tf.Session()
# #将数据转化为实数
# img_data = tf.image.convert_image_dtype(image,dtype=tf.float32)
# print(sess.run(img_data))
# sess.close()
# # 调整图像大小
# image = tf.image.resize_images(img_data,[60,160],method=0)

# sess.close()
batch_size = 20
capacity = 1000 + 3 * batch_size
# input_queue = tf.train.slice_input_producer([image, label], shuffle=False) 
example_batch, label_batch = tf.train.batch(
[image, label],batch_size= batch_size,capacity=capacity)
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    # 启动多线程处理输入数据
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    for i in range(10):
        cur_example_batch,cur_label_batch = sess.run([example_batch, label_batch])
#         cur_example_batch,cur_label_batch = sess.run([image,label])
        print(cur_example_batch,cur_label_batch)
    print(cur_example_batch.shape)


    coord.request_stop()
    coord.join(threads)

# 创建卷积神经网络进行验证码图片的识别

In [ ]:
import tensorflow as tf
sess = tf.Session()
filename = '0000'
filename2 = '0020'
names = filename.split('.')
names2 = filename2.split('.')
label = [int(i) for i in list(names[0])]
label2 = [int(i) for i in list(names2[0])]
l = tf.reshape(label,[-1])
l2 = tf.reshape(label2,[-1])
r1,r2 = sess.run([l,l2])
print(r1,r2)


In [ ]:
# 初始化权值
def weight_variables(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

# 初始化偏置值
def bias_variables(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)
# 卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")
# 池化层
def max_pool_5x5():
    return tf.nn.max_pool(x,ksize=[1,5,5,1],strides=[1,5,5,1],padding="SAME")
    
# 池化
def max_pool_4x4():
    return tf.nn.max_pool(x,ksize=[1,4,4,1],strides=[1,4,4,1],padding="SAME")

x = tf.placeholder(tf.floa32,[None,9600])
y = tf.placeholder(tf.float32,[None,4])

# width 60 height 160 channel 1
x_in = tf.reshape(x,[-1,60,160,1])

W_conv1 =  weight_variables([5,5,1,16])
b_conv1 = bias_variables([16])

h_conv1 = tf.nn.relu(conv2d(x_in,W_conv1)+b_conv1)
h_pool1 = max_pool_5x5(h_conv1)

W_conv2 =  weight_variables([5,5,16,32])
b_conv2 = bias_variables([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_4x4(h_conv2)

W_fc1 = weight_variables([3*8*32,120])
b_fc1 = bias_variables([120])

h_pool2_flat = tf.reshape(h_pool2,[-1,3*8*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
keef_pro = tf.placeholder(tf.float32)
h_fc1_drop = tf.dropout(h_fc1,keef_pro)

W_fc2 = weight_variables([120,40])
b_fc2 = bias_variables([40])

prediction = tf.matmul(h_fc1_drop,W_fc2) + b_fc2

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(y,0),tf.argmax(prediction,0))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

image = tf.decode_raw(feature['image'],tf.uint8)
image.set_shape([9600])
image = tf.cast(image,tf.float32)
label = tf.decode_raw(feature['label'],tf.int32)
label.set_shape([4])
label = tf.cast(label,tf.float32)

batch_size = 100
capacity = 1000 + 3 * batch_size
example_batch, label_batch = tf.train.batch(
[image, label],batch_size= batch_size,capacity=capacity)
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    for epoch in range(10):
        for i in rage(10):
            cur_example_batch,cur_label_batch = sess.run([example_batch, label_batch])
#         cur_example_batch,cur_label_batch = sess.run([image,label])
            print(cur_example_batch,cur_label_batch)
            sess.run(train_step,feed_dict={x:cur_example_batch,y:cur_label_batch})
    coord.request_stop()
    coord.join(threads)

